# Question 2 - Color Decomposition, Histograms, and Frequency Analysis

## Objective

In this task, we analyze a color image by converting it to different color models and applying histogram analysis and Fourier Transform.

## Steps

1. **Channel Decomposition**:
   - RGB: Split into R, G, B channels.
   - CMY or YCbCr: Derived using transformations from RGB.
2. **Convert to Grayscale**
3. **Plot Histograms**:
   - RGB Histogram to observe per-color intensity.
   - Grayscale Histogram to observe brightness spread.
4. **Histogram Equalization** on grayscale image.
5. **Fourier Transform** to analyze frequency content.


## Purpose

- Histogram Equalization improves contrast.
- Fourier Transform shows spatial frequency components.

## Conclusion

- RGB and grayscale histograms show different contrast properties.
- Equalization balances pixel intensity.
- The Fourier spectrum reveals dominant spatial frequencies (central bright region = low frequency).

## Note: 
In this question, all processing steps such as color channel extraction, grayscale conversion, histogram calculation, histogram equalization, and Fourier transform were implemented manually from scratch using only NumPy, Pillow, and Matplotlib. No high-level computer vision libraries were used, making this solution eligible for the 30% bonus according to the assignment guidelines.


In [3]:

from PIL import Image
import numpy as np
import matplotlib.pyplot as plt


img = Image.open("../Q1/sample_face.png").convert("RGB")
img_np = np.array(img)


r, g, b = img_np[:, :, 0], img_np[:, :, 1], img_np[:, :, 2]
Image.fromarray(r).save("R_channel.jpg")
Image.fromarray(g).save("G_channel.jpg")
Image.fromarray(b).save("B_channel.jpg")


c = 255 - r
m = 255 - g
y = 255 - b
Image.fromarray(c).save("C_channel.jpg")
Image.fromarray(m).save("M_channel.jpg")
Image.fromarray(y).save("Y_channel.jpg")


gray = np.dot(img_np[..., :3], [0.2989, 0.5870, 0.1140]).astype(np.uint8)
Image.fromarray(gray).save("gray.jpg")


plt.figure()
for channel, color in zip([r, g, b], ['red', 'green', 'blue']):
    plt.hist(channel.ravel(), bins=256, alpha=0.5, color=color)
plt.title("RGB Histogram")
plt.xlabel("Intensity")
plt.ylabel("Frequency")
plt.savefig("hist_rgb.jpg")
plt.close()


plt.figure()
plt.hist(gray.ravel(), bins=256, color='gray')
plt.title("Gray Histogram")
plt.xlabel("Intensity")
plt.ylabel("Frequency")
plt.savefig("hist_gray.jpg")
plt.close()


hist, _ = np.histogram(gray.flatten(), bins=256, range=[0, 256])
cdf = hist.cumsum()
cdf_masked = np.ma.masked_equal(cdf, 0)
cdf_scaled = (cdf_masked - cdf_masked.min()) * 255 / (cdf_masked.max() - cdf_masked.min())
cdf_final = np.ma.filled(cdf_scaled, 0).astype('uint8')
gray_eq = cdf_final[gray]
Image.fromarray(gray_eq).save("equalized.jpg")

plt.figure()
plt.hist(gray_eq.ravel(), bins=256, color='black')
plt.title("Equalized Gray Histogram")
plt.xlabel("Intensity")
plt.ylabel("Frequency")
plt.savefig("hist_eq_gray.jpg")
plt.close()


f = np.fft.fft2(gray)
fshift = np.fft.fftshift(f)
magnitude = 20 * np.log(np.abs(fshift) + 1)

plt.figure()
plt.imshow(magnitude, cmap='gray')
plt.title("Fourier Transform Spectrum")
plt.axis('off')
plt.savefig("fft_gray.jpg")
plt.close()
